### ACS Poverty Status by Race/Ethnicity in Phoenix Urban Villages, City of Phoenix, U.S., Maricopa County, Arizona, and Comparable Cities

For population by age 1-year data
- https://www.census.gov/data/developers/data-sets/acs-1year.html
- https://api.census.gov/data/2010/acs/acs1/variables.html
- https://api.census.gov/data/2021/acs/acs1/variables.html

For population by age 5-year data
- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

** Note: Census Block Groups (which are aggregated to Phoenix Urban Village areas) are not available until 2013. 

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import getters as get
import utilcalcs as calc
import geo_agg
from acs_poverty_vars import *

In [3]:
#Search parameters
y1 = '2021'
y0_5 = '2013' #for urban villages
y0_1 = '2010'

source5 = 'acs/acs5'
source1 = 'acs/acs1'

## Population in poverty by race/ethnicity - ACS 1 Year for larger geos

In [4]:
def year_table(source,year,cols):
    ph = get.get_phx(source,year,cols)
    mar = get.get_maricopa(source,year,cols)
    us = get.get_us(source,year,cols)
    az = get.get_az(source,year,cols)
    cit = get.get_comp_cities(source,year,cols)
    Y = pd.concat([ph,mar,us,az,cit])
    Y = Y.drop(['NAME','us','state'],axis=1)
    Y = Y[['GEO_ID']+[col for col in Y.columns if col != 'GEO_ID']]
    Y = get.clean_data(Y,Y.columns)
    Y.rename(columns=pov_rename,inplace=True)
    Y['oth_E']=Y.loc[:,oth_e].sum(axis=1)
    Y['oth_M']=Y.apply(lambda x: (calc.get_moe(x[oth_m])),axis=1)
    Y['oth_pov_E']=Y.loc[:,oth_pov_e].sum(axis=1)
    Y['oth_pov_M']=Y.apply(lambda x: (calc.get_moe(x[oth_pov_m])),axis=1)
    Y = Y.drop(oth,axis=1)
    Y = geo_agg.sumgeo_cv(Y,'GEO_ID')
    
    for col in Y.columns[1:]:
        Y.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return Y

In [5]:
Y1 = year_table(source1,y1,col_pov)
Y0 = year_table(source1,y0_1,col_pov)

In [6]:
rename_geos = {'0455000':'Phoenix', '04013':'Maricopa', '0100000US':'US', '0400000US04':'AZ',\
               '0473000':'Tempe','0465000':'Scottsdale', '0427820':'Glendale',\
               '4865000':'San Antonio', '4819000':'Dallas', '1235000':'Jacksonville'}

In [7]:
for df in [Y0,Y1]:
    df['GEO_ID'] = df.GEO_ID.map(rename_geos)

In [10]:
Y1.head(3)

,GEO_ID,bla_21E,bla_21M,bla_21C,oth_21E,oth_21M,oth_21C,ame_21E,ame_21M,ame_21C,...,his_pov_21C,tot_21E,tot_21M,tot_21C,asi_21E,asi_21M,asi_21C,whnon_21E,whnon_21M,whnon_21C
0,Phoenix,117161.0,9095.0,4.719041,634975.0,0.000000,0.00000,34366.0,5878.0,10.397638,...,6.806521,1602147.0,4965.0,0.188387,66184.0,7205.0,6.617822,636420.0,13018.0,1.243468
1,Maricopa,242999.0,8386.0,2.097898,1335655.0,0.000000,0.00000,80754.0,8162.0,6.144218,...,4.907730,4431445.0,6907.0,0.094750,192907.0,5830.0,1.837193,2312444.0,7447.0,0.195769
2,US,38536055.0,84652.0,0.133538,65225232.0,231468.801444,0.21573,3066848.0,41075.0,0.814178,...,0.658823,324173084.0,30338.0,0.005689,18833840.0,47987.0,0.154888,188537199.0,56708.0,0.018284


In [11]:
Y0.head(3)

,GEO_ID,bla_10E,bla_10M,bla_10C,oth_10E,oth_10M,oth_10C,ame_10E,ame_10M,ame_10C,...,his_pov_10C,tot_10E,tot_10M,tot_10C,asi_10E,asi_10M,asi_10C,whnon_10E,whnon_10M,whnon_10C
0,Phoenix,97946.0,7509.0,4.660468,150769.0,0.000000,0.000000,0.0,0.0,0.000000,...,5.201735,1430852.0,4556.0,0.193563,41845.0,5937.0,8.624970,683246.0,12588.0,1.119989
1,Maricopa,192854.0,4834.0,1.523744,319030.0,0.000000,0.000000,66300.0,3990.0,3.658419,...,3.752435,3778090.0,5344.0,0.085986,128263.0,4184.0,1.983008,2217970.0,3603.0,0.098751
2,US,37200236.0,54789.0,0.089533,23212045.0,143088.286876,0.374735,2465461.0,28862.0,0.711643,...,0.485491,301535021.0,22099.0,0.004455,14459936.0,31213.0,0.131221,192355136.0,40333.0,0.012746


In [8]:
with pd.ExcelWriter(f'output/poverty.xlsx') as writer:
    Y1.to_excel(writer, sheet_name="2021_1YR", index=False)
    Y0.to_excel(writer, sheet_name="2010_1YR", index=False)

#### Urban Villages

In [9]:
## WILL FILL THIS IN LATER